#IndicTrans2 English-to-Indic Training Pipeline
This notebook trains a Seq2Seq model using the `ai4bharat/indictrans2-en-indic-dist-200M` model for English-to-Indic translation.

In [2]:
!pip3 install -q datasets IndicTransToolkit evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 546.3/546.3 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 142.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 12.5 MB/s eta 0:00:00


In [ ]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface_bpcc")

HF_TOKEN = secret_value_0

!huggingface-cli login --token $HF_TOKEN

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `bpcc` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `bpcc`


In [ ]:
import wandb

user_secrets = UserSecretsClient()
my_secret = user_secrets.get_secret("wandb")

wandb.login(key=my_secret)

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: raghavdunga7 (raghavdunga7-queen-mary-university-of-london) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
import torch
from datasets import load_dataset, DatasetDict, Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from IndicTransToolkit import IndicProcessor
import re

def clean_text(text):
    text = re.sub(r'<.*?>', '', text)  #remove HTML
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)  #remove URLs
    text = re.sub(r'\s+', ' ', text).strip()  #Normalize spaces
    #DON'T lowercase or remove punctuation!
    return text


In [6]:
#configuration
selected_language = 'asm_Beng' #i change language codes here using the ISO 639-3 code
model_name = "ai4bharat/indictrans2-en-indic-dist-200M"

#load tokenizer and model with trust_remote_code=True
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, trust_remote_code=True)

#load IndicProcessor for pre/post-processing
ip = IndicProcessor(inference=True)

print(f"Model and tokenizer loaded for IndicTrans2.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenization_indictrans.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- tokenization_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


dict.SRC.json: 0.00B [00:00, ?B/s]

...

modeling_indictrans.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

Model and tokenizer loaded for IndicTrans2.


In [7]:
dataset = load_dataset("ai4bharat/BPCC", "bpcc-seed-latest", streaming=True)
samples = list(dataset[selected_language])  #like 'tam_Taml'
raw_dataset = Dataset.from_list(samples)

#Filter by source and target language tags
filtered_dataset = raw_dataset.filter(
    lambda ex: ex['src_lang'] == 'eng_Latn' and ex['tgt_lang'] == selected_language
)

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

Filter:   0%|          | 0/99581 [00:00<?, ? examples/s]

In [8]:
def tokenize_function(examples):
    src_cleaned = [clean_text(x) for x in examples['src']]
    tgt_cleaned = [clean_text(x) for x in examples['tgt']]

    src_processed = ip.preprocess_batch(
        src_cleaned, src_lang="eng_Latn", tgt_lang=selected_language
    )

    model_inputs = tokenizer(
        src_processed,
        padding="max_length",
        truncation=True,
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            tgt_cleaned,
            padding="max_length",
            truncation=True,
            max_length=128
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [9]:
tokenized_dataset = filtered_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=filtered_dataset.column_names
)

#train-validation split
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)
split_dataset = DatasetDict({
    'train': split_dataset['train'],
    'validation': split_dataset['test']
})

print("Tokenization and splitting complete.")


Parameter 'function'=<function tokenize_function at 0x7eda133baf20> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/99581 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Tokenization and splitting complete.


In [10]:
from transformers import EarlyStoppingCallback

#enhanced Training Arguments with Early Stopping
training_args = Seq2SeqTrainingArguments(
    output_dir="./results_with_early_stopping",

    #Batch sizes
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,  #effective batch size = 16

    #training schedule
    num_train_epochs=3,  #set higher since early stopping will control actual epochs
    learning_rate=4e-5,
    warmup_steps=200,  #learning rate warmup

    #EVALUATION STRATEGY (critical for early stopping!)
    eval_strategy="steps",
    eval_steps=2000,  #evaluate every 300 steps (more frequent = better early stopping)

    #EARLY STOPPING CONFIGURATION
    load_best_model_at_end=True,  #load best checkpoint at end
    metric_for_best_model="eval_loss",  #can also use "eval_bleu" if you add custom metrics
    greater_is_better=False,  #For loss (lower is better), set True for BLEU

    #checkpointing (important for early stopping)
    save_strategy="steps",
    save_steps=2000,  #save at same frequency as evaluation
    save_total_limit=1,  #Keep best 1 checkpoint

    #logging
    logging_steps=200,
    logging_strategy="steps",

    #memory optimization
    #fp16=torch.cuda.is_available(),
    dataloader_pin_memory=True,
    remove_unused_columns=False,

    #generation parameters
    predict_with_generate=True,
    generation_max_length=256,
    generation_num_beams=4,
    #generation_do_sample=False,
    weight_decay=0.01,  #regularization
    optim="adamw_torch",  #Use AdamW optimizer

    #reporting
    report_to="wandb" if 'wandb' in globals() else None,
    run_name="indictrans2-malayalam-early-stopping"
)

#create Early Stopping Callback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,  #stop if no improvement for 3 evaluations
    early_stopping_threshold=0.001,  #minimum improvement threshold (0.1% improvement required)
)

In [ ]:
#trainer setup
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset['train'],
    eval_dataset=split_dataset['validation'],
    tokenizer=tokenizer
)

trainer.train()


/tmp/ipython-input-3927751075.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss,Validation Loss
2000,0.573300,0.478049
4000,0.372500,0.317624
6000,0.293100,0.270998
8000,0.267500,0.248056
10000,0.243600,0.236241
12000,0.223900,0.227595


In [ ]:
from tqdm import tqdm
from datasets import load_dataset, Dataset
from evaluate import load
import torch

#fIXED EVALUATION CODE
print("=== FIXED FLORES-200/BPCC MODEL EVALUATION ===")

def load_flores_evaluation_data(target_language='asm_Beng', num_samples=1012):
    """
    Load FLORES-200 data with proper error handling
    """
    print(f"Loading FLORES-200 for {target_language}...")

    #method 1: Try direct FLORES-200 loading (may work in some environments)
    try:
        flores_dataset = load_dataset(
            "facebook/flores",
            name="all",  #load all languages
            split="devtest",
            trust_remote_code=False  #Don't use custom scripts
        )

        #check if data exists for your language
        if f'sentence_{target_language}' in flores_dataset.column_names:
            clean_samples = []
            for i, sample in enumerate(flores_dataset):
                if i >= num_samples:
                    break

                clean_sample = {
                    'src': sample['sentence_eng_Latn'].strip(),
                    'tgt': sample[f'sentence_{target_language}'].strip(),
                    'src_lang': 'eng_Latn',
                    'tgt_lang': target_language
                }

                if (len(clean_sample['src']) > 10 and
                    len(clean_sample['tgt']) > 10):
                    clean_samples.append(clean_sample)

            print(f"Loaded {len(clean_samples)} FLORES-200 samples")
            return Dataset.from_list(clean_samples)

    except Exception as e:
        print(f"FLORES-200 failed: {e}")

    #method 2: Try alternative FLORES repo
    try:
        print("💡 Trying alternative FLORES-200 repository...")
        flores_dataset = load_dataset(
            "gsarti/flores_101",
            name=f"eng-{target_language[:3]}",  #Use 3-letter code
            split="devtest"
        )

        clean_samples = []
        for sample in flores_dataset:
            clean_sample = {
                'src': sample['sentence_eng'].strip(),
                'tgt': sample[f'sentence_{target_language[:3]}'].strip(),
                'src_lang': 'eng_Latn',
                'tgt_lang': target_language
            }

            if (len(clean_sample['src']) > 10 and
                len(clean_sample['tgt']) > 10):
                clean_samples.append(clean_sample)

        print(f"Loaded {len(clean_samples)} FLORES samples from alternative repo")
        return Dataset.from_list(clean_samples)

    except Exception as e:
        print(f"Alternative FLORES failed: {e}")

    print("FLORES-200 not available, will use BPCC...")
    return None

def evaluate_model_complete(model, tokenizer, ip, target_language='asm_Beng', batch_size=8, max_samples=1012):
    """
    Complete evaluation function that works with both FLORES and BPCC
    """

    #1. Try FLORES-200 first
    eval_dataset = load_flores_evaluation_data(target_language, num_samples=max_samples)

    #2. If FLORES fails, use BPCC
    if eval_dataset is None:
        print("Loading BPCC evaluation data...")
        try:
            dataset = load_dataset(
                "ai4bharat/BPCC",
                "bpcc-seed-latest",
                streaming=True
            )

            clean_samples = []
            count = 0

            for sample in dataset[target_language]:
                if (sample['src_lang'] == 'eng_Latn' and
                    sample['tgt_lang'] == target_language and
                    len(sample['src'].strip()) > 10 and
                    len(sample['tgt'].strip()) > 10):

                    clean_samples.append({
                        'src': sample['src'].strip(),
                        'tgt': sample['tgt'].strip(),
                        'src_lang': 'eng_Latn',
                        'tgt_lang': target_language
                    })
                    count += 1
                    if count >= max_samples:
                        break

            if len(clean_samples) == 0:
                print("No valid BPCC samples found!")
                return None

            eval_dataset = Dataset.from_list(clean_samples)
            print(f"Loaded {len(clean_samples)} clean BPCC samples")

        except Exception as e:
            print(f"BPCC loading failed: {e}")
            return None

    #3. Run evaluation on whatever dataset we have
    model.eval()

    predictions = []
    references = []
    source_texts = []

    print(f"Running evaluation on {target_language}...")

    total_samples = min(max_samples, len(eval_dataset))

    for i in tqdm(range(0, total_samples, batch_size), desc="Evaluating"):
        batch_end = min(i + batch_size, total_samples)
        batch_samples = eval_dataset[i:batch_end]

        #Handle both single samples and batches
        if isinstance(batch_samples['src'], str):
            src_texts = [batch_samples['src']]
            tgt_texts = [batch_samples['tgt']]
        else:
            src_texts = batch_samples['src']
            tgt_texts = batch_samples['tgt']

        #Preprocess sources
        try:
            src_processed = ip.preprocess_batch(
                src_texts, src_lang="eng_Latn", tgt_lang=target_language
            )
        except:
            #fallback if batch preprocessing fails
            src_processed = []
            for text in src_texts:
                processed = ip.preprocess_batch([text], src_lang="eng_Latn", tgt_lang=target_language)
                src_processed.extend(processed)

        #tokenize
        inputs = tokenizer(
            src_processed,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=256
        ).to(model.device)

        #generate
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=128,
                num_beams=4,
                early_stopping=True,
                do_sample=False,
                length_penalty=1.0
            )

        #decode
        batch_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)

        #Post-process
        try:
            batch_preds = ip.postprocess_batch(batch_preds, lang=target_language)
        except:
            #fallback if batch postprocessing fails
            processed_preds = []
            for pred in batch_preds:
                processed = ip.postprocess_batch([pred], lang=target_language)
                processed_preds.extend(processed)
            batch_preds = processed_preds

        predictions.extend(batch_preds)
        references.extend(tgt_texts)
        source_texts.extend(src_texts)

    #4. Show sample results
    print(f"\n=== EVALUATION SAMPLES ({target_language}) ===")
    for i in range(min(3, len(predictions))):
        print(f"\nExample {i+1}:")
        print(f"Source:     {source_texts[i]}")
        print(f"Prediction: {predictions[i]}")
        print(f"Reference:  {references[i]}")

    #5. Compute metrics
    print(f"\n=== COMPUTING METRICS ({target_language}) ===")

    #filter valid pairs
    valid_pairs = []
    for pred, ref, src in zip(predictions, references, source_texts):
        if (pred and pred.strip() and
            ref and ref.strip() and
            len(pred.strip()) > 2 and
            len(ref.strip()) > 2):
            valid_pairs.append((pred.strip(), ref.strip(), src.strip()))

    if not valid_pairs:
        print("No valid prediction-reference pairs found!")
        return None

    valid_preds, valid_refs, valid_sources = zip(*valid_pairs)
    print(f"Valid pairs for evaluation: {len(valid_pairs)}")

    #compute metrics
    try:
        bleu_metric = load("bleu")
        meteor_metric = load("meteor")
        chrf_metric = load("chrf")

        #BLEU
        bleu_result = bleu_metric.compute(
            predictions=list(valid_preds),
            references=[[r] for r in valid_refs],
            smooth=True
        )
        bleu_score = bleu_result["bleu"] * 100

        #mETEOR
        meteor_score = meteor_metric.compute(
            predictions=list(valid_preds),
            references=list(valid_refs)
        )["meteor"] * 100

        #cHRF++
        chrf_score = chrf_metric.compute(
            predictions=list(valid_preds),
            references=list(valid_refs)
        )["score"]

        #results
        print(f"\n EVALUATION RESULTS ({target_language}):")
        print(f"{'='*50}")
        print(f"BLEU:     {bleu_score:.2f}")
        print(f"METEOR:   {meteor_score:.2f}")
        print(f"CHRF++:   {chrf_score:.2f}")
        print(f"Samples:  {len(valid_pairs)}")
        print(f"{'='*50}")

        return {
            'bleu': bleu_score,
            'meteor': meteor_score,
            'chrf': chrf_score,
            'num_samples': len(valid_pairs),
            'dataset_used': 'FLORES-200' if 'flores' in str(type(eval_dataset)).lower() else 'BPCC'
        }

    except Exception as e:
        print(f"Metric computation failed: {e}")
        print("Trying simple BLEU computation...")

        try:
            from collections import Counter
            import math

            #simple BLEU implementation as fallback
            def simple_bleu(pred, ref, n=4):
                pred_words = pred.split()
                ref_words = ref.split()

                if len(pred_words) == 0 or len(ref_words) == 0:
                    return 0.0

                #Brevity penalty
                bp = min(1.0, math.exp(1 - len(ref_words) / len(pred_words)))

                #N-gram precisions
                precisions = []
                for i in range(1, min(n + 1, len(pred_words) + 1)):
                    pred_ngrams = Counter([tuple(pred_words[j:j+i]) for j in range(len(pred_words) - i + 1)])
                    ref_ngrams = Counter([tuple(ref_words[j:j+i]) for j in range(len(ref_words) - i + 1)])

                    overlap = sum((pred_ngrams & ref_ngrams).values())
                    total = sum(pred_ngrams.values())

                    if total == 0:
                        precisions.append(0.0)
                    else:
                        precisions.append(overlap / total)

                if all(p > 0 for p in precisions):
                    score = bp * math.exp(sum(math.log(p) for p in precisions) / len(precisions))
                else:
                    score = 0.0

                return score * 100

            #compute simple BLEU for each pair and average
            bleu_scores = [simple_bleu(pred, ref) for pred, ref in zip(valid_preds, valid_refs)]
            avg_bleu = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0.0

            print(f"\n FALLBACK RESULTS ({target_language}):")
            print(f"{'='*50}")
            print(f"Simple BLEU: {avg_bleu:.2f}")
            print(f"Samples:     {len(valid_pairs)}")
            print(f"{'='*50}")

            return {
                'bleu': avg_bleu,
                'meteor': None,
                'chrf': None,
                'num_samples': len(valid_pairs),
                'dataset_used': 'FLORES-200' if 'flores' in str(type(eval_dataset)).lower() else 'BPCC'
            }

        except Exception as e2:
            print(f"Even fallback BLEU failed: {e2}")
            return None

#mAIN EVALUATION FUNCTION
def run_evaluation(model, tokenizer, ip, target_language='asm_Beng'):
    """
    Run the complete evaluation (FLORES-200 or BPCC fallback)
    """
    print(f" Starting Evaluation for {target_language}")
    print("="*60)

    results = evaluate_model_complete(
        model=model,
        tokenizer=tokenizer,
        ip=ip,
        target_language=target_language,
        batch_size=4,  #smaller batch for stability
        max_samples=1012  #FLORES devtest size
    )

    if results:
        print("\n Evaluation Complete!")
        print("="*60)
        print(f"Dataset used: {results.get('dataset_used', 'Unknown')}")
        return results
    else:
        print("\n Evaluation completely failed")
        return None

#USAGE:
#results = run_evaluation(model, tokenizer, ip, target_language='guj_Gujr')

In [ ]:
results = run_evaluation(model, tokenizer, ip, target_language='asm_Beng')